In [ ]:
import os
import glob
import random
import tensorflow as tf
from keras_video import VideoFrameGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout,Conv2D, BatchNormalization,MaxPool2D, GlobalMaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
# from tensorflow.python.keras.engine.training import Model

seed =88
tf.random.set_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [ ]:
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('D://gitcourse//videos/*')]
classes.sort()

# some global params
SIZE = (128, 128)
CHANNELS = 3
NBFRAME = 20
BS = 32

# pattern to get videos and classes
glob_pattern='D:\\gitcourse\\videos\\{classname}\\*.avi'

# for data augmentation
data_aug = ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=False,
#     rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)

In [ ]:
valid = train.get_validation_generator()

In [ ]:
def build_convnet(shape=(SIZE[0], SIZE[0], CHANNELS)):
    momentum = .9
    model = Sequential()
    
    model.add(Conv2D(16, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    
    model.add(Conv2D(16, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(GlobalMaxPool2D())
    return model

def action_model(shape=(NBFRAME, SIZE[0], SIZE[0], CHANNELS), nbout=3):

    convnet = build_convnet(shape[1:])    
    # final model
    model = Sequential()
    # add the convnet 
    model.add(TimeDistributed(convnet, input_shape=shape))
        
    model.add(GRU(
                  units=128,
                  activation='tanh',
                  recurrent_activation = 'sigmoid',
                  recurrent_dropout=0.0,
                  unroll=False,
                  use_bias =True,
                  reset_after =True,
                 
                 ))

    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) 
model = action_model(INSHAPE, len(classes))
optimizer = Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)
model.summary()

In [ ]:
EPOCHS=30

callbacks = [
    ReduceLROnPlateau(verbose=1),
    ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]

history = model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(0,30)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='Validation loss')
plt.plot(loss_train)
plt.plot(loss_val)
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_train = history.history['acc']
acc_val = history.history['val_acc']
epochs = range(0,30)
plt.plot( acc_train, 'g', label='Training accuracy')
plt.plot( acc_val, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
train_loss, train_accuracy = model.evaluate_generator(train)
train_accuracy

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid)
valid_accuracy